## README: This is the final notebook. Use this notebook to paste the final code.
Do not test code here :)

### Pre Processing of Tweets

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta, datetime
from dateutil import parser
import re
import emoji
import string

In [20]:
tweets = pd.read_csv('raw_trump_tweets.csv')

In [21]:
# >= 20th jan 2017, <= 31st Dec 2018
tweets['date']= pd.to_datetime(tweets['date'])
tweets = tweets[tweets['date'] >= np.datetime64('2017-01-20')]
tweets = tweets[tweets['date'] <= np.datetime64('2018-12-31')]


In [22]:
def process_and_remove_duplicates(tweets, date):
    duplicated_rows = tweets[tweets['date'] == date]
    id = max(duplicated_rows['id'])
    max_favorite = max(duplicated_rows['favorites'])
    max_retweet = max(duplicated_rows['retweets'])
    combined_tweet = ""
    device = duplicated_rows['device'].iloc[0]
    for index, row in duplicated_rows.iterrows():
        combined_tweet = row['text'] + ' ' + combined_tweet.strip('.')
    tweets = tweets[tweets.date != date]
    row = {'id': id, 'text': combined_tweet, 'isRetweet': 'f', 'isDeleted': 'f', 'device': device, 'favorites': max_favorite, 'retweets': max_retweet, 'date': date}
    tweets = tweets.append(row, ignore_index = True)   
    print(f"Combined {len(duplicated_rows)} tweets on {date}")
    return tweets

In [23]:
dates = tweets["date"]
duplicated = tweets[dates.isin(dates[dates.duplicated()])]
duplicated_dates = duplicated['date'].unique()

for date in duplicated_dates:
    tweets = process_and_remove_duplicates(tweets, date)

Combined 2 tweets on 2018-11-21T22:18:00.000000000
Combined 2 tweets on 2018-09-10T21:59:00.000000000
Combined 2 tweets on 2018-08-14T01:37:00.000000000
Combined 2 tweets on 2018-05-31T10:56:00.000000000
Combined 2 tweets on 2018-02-01T22:37:00.000000000
Combined 2 tweets on 2018-03-09T17:50:00.000000000
Combined 2 tweets on 2017-12-29T12:50:00.000000000
Combined 2 tweets on 2017-12-29T12:48:00.000000000
Combined 3 tweets on 2017-12-28T22:17:00.000000000
Combined 2 tweets on 2017-12-22T20:47:00.000000000
Combined 2 tweets on 2017-12-16T03:09:00.000000000
Combined 2 tweets on 2017-11-18T13:49:00.000000000
Combined 2 tweets on 2017-11-15T10:22:00.000000000
Combined 2 tweets on 2017-07-11T10:53:00.000000000
Combined 2 tweets on 2017-07-11T10:37:00.000000000
Combined 2 tweets on 2017-10-28T21:09:00.000000000
Combined 2 tweets on 2017-10-26T01:47:00.000000000
Combined 2 tweets on 2017-10-14T11:08:00.000000000
Combined 2 tweets on 2017-05-10T10:31:00.000000000
Combined 2 tweets on 2017-09-30

Combined 2 tweets on 2018-06-15T11:56:00.000000000
Combined 4 tweets on 2018-06-15T11:55:00.000000000
Combined 2 tweets on 2018-12-06T08:53:00.000000000
Combined 2 tweets on 2018-06-14T15:09:00.000000000
Combined 2 tweets on 2018-06-14T15:08:00.000000000
Combined 2 tweets on 2018-06-13T09:40:00.000000000
Combined 2 tweets on 2018-12-06T20:40:00.000000000
Combined 2 tweets on 2018-09-06T20:58:00.000000000
Combined 2 tweets on 2018-09-06T20:56:00.000000000
Combined 2 tweets on 2018-06-06T00:40:00.000000000
Combined 2 tweets on 2018-05-30T12:47:00.000000000
Combined 4 tweets on 2018-05-25T12:04:00.000000000
Combined 2 tweets on 2018-05-23T01:13:00.000000000
Combined 2 tweets on 2018-05-16T18:40:00.000000000
Combined 3 tweets on 2018-05-16T13:09:00.000000000
Combined 2 tweets on 2018-08-05T20:33:00.000000000
Combined 3 tweets on 2018-04-21T13:10:00.000000000
Combined 3 tweets on 2018-04-21T12:17:00.000000000
Combined 2 tweets on 2018-04-20T10:34:00.000000000
Combined 2 tweets on 2018-04-19

In [24]:
## tweets that are Trump's
tweets1 = tweets[tweets['isRetweet'] == 'f'].reset_index(drop=True)
tweets1 = tweets1[tweets1['text'].str.contains('RT @') == False]
tweets1['cleaned_text'] = tweets1['text'].apply(lambda x: re.sub(r'https?:\/\/\S*', '', x, flags=re.MULTILINE))
tweets1 = tweets1[tweets1['cleaned_text'].str.strip() != '']

tweets2 = tweets1.loc[:, ['id', 'cleaned_text', 'favorites', 'retweets', 'date']]
tweets2['date_new'] = [i + timedelta(hours = 8) for i in tweets2['date']]
tweets2['date_part'] = [i.date() for i in tweets2['date_new']]
tweets2['time_part'] = [i.time() for i in tweets2['date_new']]
tweets2['hour'] = [int(str(i).split(":")[0]) for i in tweets2['time_part']]
tweets2['year'] = [int(str(i).split("-")[0]) for i in tweets2['date_part']]
tweets2['month'] = [int(str(i).split("-")[1]) for i in tweets2['date_part']]


In [25]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_mentions(string):
    return re.sub("@[A-Za-z0-9]+","",string)
    

In [26]:
tweets2["cleaned_text"] = tweets2["cleaned_text"].str.lower()
tweets2["cleaned_text2"] = tweets2.cleaned_text.apply(remove_emoji)
tweets2.cleaned_text2 = tweets2.cleaned_text.apply(remove_mentions)

In [27]:
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'http\S+', '', text)
    return text

# Apply the function above and get tweets free of emoji's
call_emoji_free = lambda x: give_emoji_free_text(x)

# Apply `call_emoji_free` which calls the function to remove all emoji's
tweets2['emoji_free_tweets'] = tweets2['cleaned_text'].apply(call_emoji_free)

#Create a new column with url free tweets
tweets2['url_free_tweets'] = tweets2['emoji_free_tweets'].apply(url_free_text)
df = tweets2

In [29]:
for i in range(0,5088):
    df.iloc[i,-3] = re.sub('[%s]' % re.escape(string.punctuation), '',df.iloc[i,-1])
    df.iloc[i,-3] = re.sub("@[A-Za-z0-9]+","",df.iloc[i,-3])
    df.iloc[i,-3]=re.sub('\w*\d\w*', '', df.iloc[i,-3])
    df.iloc[i,-3] = df.iloc[i,-3].strip("'")
    df.iloc[i,-3]=df.iloc[i,-3].replace("#", "")
    df.iloc[i,-3].strip('"') 

In [31]:
df.to_csv("trump_tweets_cleaned_2.csv", index=False, header=True)

### Pre Processing of Stock Data

In [ ]:
# Datasets are combined after pre processing! (else kernel crashes)
stock_price_2017 = pd.read_csv('yr2017.csv')
stock_price_2018 = pd.read_csv('yr2018.csv')

In [ ]:
# Drop unwanted columns 
stock_price_2017.drop(columns=['SYM_ROOT', 'SYM_SUFFIX'], inplace=True)
stock_price_2018.drop(columns=['SYM_ROOT', 'SYM_SUFFIX'], inplace=True)

In [ ]:
# Change to datetime objects
stock_price_2017['TIME_M'] = pd.to_datetime(stock_price_2017['TIME_M'], format='%H:%M:%S.%f')
stock_price_2018['TIME_M'] = pd.to_datetime(stock_price_2018['TIME_M'], format='%H:%M:%S.%f')

In [ ]:
# Remove seconds, milliseconds from data
stock_price_2017['TIME_M'] = stock_price_2017['TIME_M'].dt.floor('Min').dt.time
stock_price_2018['TIME_M'] = stock_price_2018['TIME_M'].dt.floor('Min').dt.time

In [ ]:
# Get minute to minute data (one entry for each minute)
stock_price_2017.drop_duplicates(subset=['TIME_M','DATE'],inplace=True)
stock_price_2018.drop_duplicates(subset=['TIME_M', 'DATE'],inplace=True)

In [ ]:
# Export to csv, will be used for preprocessing later!
stock_price = pd.concat([stock_price_2017,stock_price_2018], ignore_index=True, sort=False)
stock_price.to_csv("stock_price.csv", index=False, header=True)

In [ ]:
## Interpolation of missing values
idx = pd.date_range(stock_price.iloc[0, 0], stock_price.iloc[-1, 0], freq='1min')
stock_price.index = pd.DatetimeIndex(stock_price.loc[:, 'DATE_TIME_M'])
stock_price = stock_price.reindex(idx, fill_value='NaN')
stock_price['DATE_TIME_M'] = stock_price.index

In [ ]:
stock_price = stock_price.astype({'PRICE': 'float'})
stock_price['PRICE'].interpolate(method='linear', inplace=True)

In [ ]:
stock_price.to_csv("stock_price.csv", index=False, header=True)